In [1]:
include("optim_lib.jl")
include("lib_func.jl")
using Optim


# Constants and Parameters
d1, d2, ds = 3, 4, 1  # Degree range and step
const n, a, b = 4, 10, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .9 , 2 / 10  # Sampling parameters
f = shubert_4d # Objective function

shubert_4d (generic function with 1 method)

In [2]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.2)
# precis_poly_approx = precise_gen(f, n, d1, d2, ds, delta, alph, C, 0.2) ## test ##

dimension Vector space: 35
sample size: 256
dimension Vector space: 70
sample size: 256


2-element Vector{Vector{Float64}}:
 [0.06249014079223168, -0.5357054864177533, 0.25841609771318813, 0.25363619681339755, -0.5357054864177538, 9.184820662582268, -4.4306164002380015, 0.25841609771318813, -4.430616400238001, 0.25363619681339705  …  2.358198851602585e-16, 9.184820662582272, -1.6043023390711187e-15, 1.6299302579437611e-16, -4.430616400238006, 0.2584160977131883, 2.3322545246911183e-16, 2.282898679726866e-16, -4.430616400238002, 0.2536361968133976]
 [-3.3457849565942133, -0.9852754849692609, -0.5722809424725449, 1.3389941845524993, -7.054336497542864e15, -0.00047785360292995627, 9.184820662582268, -4.4306164002380015, -4.34866366004288, 1.2473878080973526  …  -0.0, -4.430616400238001, 8.881784197001252e-16, 4.85722573273506e-17, 2.137261293084298, 0.004453871042878957, 1.6361926408190708e-16, 4.473256443070617e-16, -4.348663660042881, 1.619572523612494e15]

In [3]:
println(coeffs_poly_approx[2])
println(precis_poly_approx[2]) 

[-3.3457849565942133, -0.9852754849692609, -0.5722809424725449, 1.3389941845524993, -7.054336497542864e15, -0.00047785360292995627, 9.184820662582268, -4.4306164002380015, -4.34866366004288, 1.2473878080973526, -4.430616400238001, 2.137261293084298, -1.0385176132849978, -4.348663660042881, 8.398415901470575e15, -2.2897263147736564, -7.383114653332456e-16, 4.955666893126103e-16, -1.1737622203062085e-16, 1.8468125200983834e-16, 2.348854131586621e-16, -4.381993218976211e-16, -4.724155054208991e-16, -3.33066907387547e-16, -1.9428902930940242e-16, -2.982591788119837, 4.633553796330258e-16, -5.833207410171614e-16, 1.296761656324482e-16, 1.0453935102812921e-16, 1.2160197199843972e-17, 4.488217069315108, 1.416359112974944e-16, -3.8712719742960566e-16, -2.752286226124538e16, -0.43249078757993376, 5.500285032349972e-16, -3.112153383117372e-16, 7.45931094670027e-17, -1.6653345369377343e-16, -2.3592239273284586e-16, -7.91033905045424e-16, 1.5937800208088722e-32, -4.440892098500626e-16, 2.220446049

UndefVarError: UndefVarError: `precis_poly_approx` not defined

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [5]:


using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames, Optim

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, h_t, col = main_4d(n, d1, d2, ds, coeffs_poly_approx) # main_4d is in hom_solve.jl
df = DataFrame(x=C * h_x, y=C * h_y, z=C * h_z, t=C * h_t, col=col)
df[!, :result] = [f([df.x[i], df.y[i], df.z[i], df.t[i]]) for i in 1:nrow(df)]
df[!, :distance_to_minima] = zeros(nrow(df))
df[!, :steps] = zeros(Int, nrow(df))
df[!, :converged] = falses(nrow(df))

println(df)

In [6]:


for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)

    distance = norm(x0 - minimizer)

    df.distance_to_minima[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged

    println("Initial point: ", x0)
    println("Minimizer: ", minimizer)
    println("Distance to minima: ", distance)
    println("Number of steps: ", steps)
    println("Converged: ", converged)
    println(summary(res))
end

println(df)

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 8    Time: 0:00:01
  mixed_volume:  16


Tracking 16 paths...  12%|███▊                          |  ETA: 0:01:11



Tracking 16 paths... 100%|██████████████████████████████| Time: 0:00:11
  # paths tracked:                  16
  # non-singular solutions (real):  16 (4)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         16 (4)
Degree: 3
Number of solutions: 4
Tracking 81 paths...   2%|▊                             |  ETA: 0:03:06



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:04
  # paths tracked:                  81
  # non-singular solutions (real):  81 (81)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         81 (81)
Degree: 4
Number of solutions: 81
85×9 DataFrame
 Row │ x             y             z             t             col    result       distance_to_minima  steps  converged 
     │ Float64       Float64       Float64       Float64       Int64  Float64      Float64             Int64  Bool      
─────┼────────────────────────────────────────